## Data preprocessing and analysis PART I

### Libraries:

In [150]:
import matplotlib.pyplot as plt
import pandas as pd
import calendar
from scipy import stats
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import calplot
import json
import geopandas as gpd
import statistics

### Loading housing data:
https://data.cambridgeshireinsight.org.uk/dataset/house-sales-and-prices

In [151]:
df1 = pd.read_csv('../data/ons-existing-homes-count-corrected_0.csv').sort_values('Area')
df2 = pd.read_csv('../data/ons-existing-homes-median-price-corrected_0_0.csv').sort_values('Area')
df3 = pd.read_csv('../data/ons-new-build-count-corrected_0.csv').sort_values('Area')
df4 = pd.read_csv('../data/ons-new-build-median-price-corrected_0.csv').sort_values('Area')

In [152]:
df2

,ONS code,Area,Q4-1995,Q1-1996,Q2-1996,Q3-1996,Q4-1996,Q1-1997,Q2-1997,Q3-1997,...,Q3-2013,Q4-2013,Q1-2014,Q2-2014,Q3-2014,Q4-2014,Q1-2015,Q2-2015,Q3-2015,Q4-2015
1,E07000008,Cambridge,69500,69950,70000,72000,72500,73000,75000,78500,...,290000,295000,295000,304075,315000,327350,337000,350000,370000,380000
2,E07000009,East Cambridgeshire,56500,56500,57000,57000,57500,58000,58500,59000,...,196011,195750,197000,200000,204050,215000,220000,226000,229950,240000
3,E07000010,Fenland,40000,40000,40000,40000,40000,40000,41500,43000,...,127000,130000,135000,137000,139995,140000,141000,145000,147500,150000
6,E07000201,Forest Heath,49000,49000,48000,47975,48000,49000,50000,52500,...,149999,150000,151000,155000,160000,164950,168500,170000,175000,179500
4,E07000011,Huntingdonshire,54000,54000,54000,54625,54875,55000,56000,57000,...,177000,178000,179995,184500,190000,195000,198000,200000,210000,215000
0,E06000031,Peterborough,40500,40000,40000,40000,41000,41500,42000,43000,...,125000,125000,129250,132750,135100,138000,140000,143500,145000,148000
5,E07000012,South Cambridgeshire,69950,70000,70000,73125,74000,74000,75500,77500,...,250000,250000,250000,250000,259875,266500,275000,284000,300000,315000
7,E07000204,St Edmundsbury,53500,53500,53625,54000,54995,55000,56625,57000,...,186000,190000,192500,196000,208000,215000,215000,225000,227748,231000


### Data preprocessing

Since there are only data from december 1995 we remove all data from 2015 and thery only have data from 1996-2015

In [153]:
df1.drop(columns="Q4-1995", inplace=True)
df2.drop(columns="Q4-1995", inplace=True)
df3.drop(columns="Q4-1995", inplace=True)
df4.drop(columns="Q4-1995", inplace=True)

According to wikipedia: "St Edmundsbury and Forest Heath were merged on 1 April 2019 to form the new West Suffolk district"
https://en.wikipedia.org/wiki/Borough_of_St_Edmundsbury \
Therfore we merge the data for these two areas in order to compare over the years. 
We add the amount of houses sold in data 1 and 3 and find the median price in data 2 and 4.

In [154]:
areas_to_merge = ['Forest Heath', 'St Edmundsbury']
new_area_name = 'West Suffolk'
new_ons_code = 'E07000245'

In [155]:
merge_df = df1[df1['Area'].isin(areas_to_merge)]
merged_row = merge_df.select_dtypes(include=[np.number]).sum()
merged_row_df = pd.DataFrame([merged_row], columns=merge_df.columns.drop(['ONS code', 'Area']))
merged_row_df['ONS code'] = new_ons_code
merged_row_df['Area'] = new_area_name

df1 = df1[~df1['Area'].isin(areas_to_merge)]
df1 = pd.concat([df1, merged_row_df], ignore_index=True)

df1

,ONS code,Area,Q1-1996,Q2-1996,Q3-1996,Q4-1996,Q1-1997,Q2-1997,Q3-1997,Q4-1997,...,Q3-2013,Q4-2013,Q1-2014,Q2-2014,Q3-2014,Q4-2014,Q1-2015,Q2-2015,Q3-2015,Q4-2015
0,E07000008,Cambridge,1438,1458,1546,1718,1829,1979,2095,2102,...,1293,1320,1383,1412,1366,1378,1352,1327,1345,1310
1,E07000009,East Cambridgeshire,940,964,1014,1139,1172,1268,1362,1396,...,1166,1264,1337,1423,1462,1448,1428,1405,1308,1312
2,E07000010,Fenland,1214,1262,1296,1389,1491,1610,1763,1875,...,1379,1439,1562,1649,1684,1708,1617,1563,1597,1660
3,E07000011,Huntingdonshire,2575,2713,2910,3157,3347,3443,3485,3364,...,2419,2699,2926,3076,3328,3296,3182,3118,2975,2938
4,E06000031,Peterborough,2212,2194,2356,2530,2628,2808,2921,3022,...,1927,2021,2162,2218,2407,2460,2428,2423,2453,2488
5,E07000012,South Cambridgeshire,1773,1789,1928,2172,2280,2442,2563,2519,...,1986,2054,2195,2311,2380,2416,2351,2231,2133,2151
6,E07000245,West Suffolk,2104,2204,2393,2638,2751,2972,3133,3182,...,2431,2579,2731,2890,3022,3114,2982,2881,2842,2895


In [156]:
merge_df = df3[df3['Area'].isin(areas_to_merge)]
merged_row = merge_df.select_dtypes(include=[np.number]).sum()
merged_row_df = pd.DataFrame([merged_row], columns=merge_df.columns.drop(['ONS code', 'Area']))
merged_row_df['ONS code'] = new_ons_code
merged_row_df['Area'] = new_area_name

df3 = df3[~df3['Area'].isin(areas_to_merge)]
df3 = pd.concat([df3, merged_row_df], ignore_index=True)
df3

,ONS code,Area,Q1-1996,Q2-1996,Q3-1996,Q4-1996,Q1-1997,Q2-1997,Q3-1997,Q4-1997,...,Q3-2013,Q4-2013,Q1-2014,Q2-2014,Q3-2014,Q4-2014,Q1-2015,Q2-2015,Q3-2015,Q4-2015
0,E07000008,Cambridge,169,140,169,198,240,239,230,176,...,595,669,724,703,606,580,530,457,363,255
1,E07000009,East Cambridgeshire,324,389,445,483,488,486,468,453,...,151,136,143,125,122,124,109,104,91,80
2,E07000010,Fenland,252,215,209,199,211,220,254,262,...,114,116,121,126,120,126,117,92,86,78
3,E07000011,Huntingdonshire,589,578,591,616,643,668,673,639,...,345,411,428,457,444,388,385,353,319,281
4,E06000031,Peterborough,266,278,328,365,370,413,411,428,...,485,561,626,651,669,644,614,562,572,501
5,E07000012,South Cambridgeshire,339,354,397,396,430,485,550,558,...,372,388,417,445,451,453,420,403,372,312
6,E07000245,West Suffolk,477,492,527,545,568,599,577,541,...,254,239,283,291,339,379,360,387,360,330


In [157]:
#merge 'Forest Heath' and 'St Edmundsbury' in data frames with prices - df2 and df4

def process_df(df, areas_to_merge, new_area_name, new_ons_code):
    merge_df = df[df['Area'].isin(areas_to_merge)]
    
    merged_row = merge_df.select_dtypes(include=[np.number]).mean()
    merged_row_df = pd.DataFrame([merged_row], columns=merge_df.columns.drop(['ONS code', 'Area']))
    merged_row_df['ONS code'] = new_ons_code
    merged_row_df['Area'] = new_area_name
    df = df[~df['Area'].isin(areas_to_merge)]
    return pd.concat([df, merged_row_df], ignore_index=True)

df2 = process_df(df2, areas_to_merge, new_area_name, new_ons_code)
df4 = process_df(df4, areas_to_merge, new_area_name, new_ons_code)

Reshaping data:

In [164]:
df1_flip = df1.melt(id_vars=['ONS code', 'Area'], var_name='Quarter', value_name='Houses Sold')
df2_flip = df2.melt(id_vars=['ONS code', 'Area'], var_name='Quarter', value_name='Price')
df3_flip = df3.melt(id_vars=['ONS code', 'Area'], var_name='Quarter', value_name='Houses Sold')
df4_flip = df4.melt(id_vars=['ONS code', 'Area'], var_name='Quarter', value_name='Price')

Extracting features:

In [165]:
# Extract year and quarter
df1_flip[['Quarter', 'Year']] = df1_flip['Quarter'].str.split('-', expand=True)
df2_flip[['Quarter', 'Year']] = df2_flip['Quarter'].str.split('-', expand=True)
df3_flip[['Quarter', 'Year']] = df3_flip['Quarter'].str.split('-', expand=True)
df4_flip[['Quarter', 'Year']] = df4_flip['Quarter'].str.split('-', expand=True)

# Convert Year to integer
df1_flip['Year'] = df1_flip['Year'].astype(int)
df2_flip['Year'] = df2_flip['Year'].astype(int)
df3_flip['Year'] = df3_flip['Year'].astype(int)
df4_flip['Year'] = df4_flip['Year'].astype(int)

In [166]:
# Combine data frames df1 and df2 for resale market
df_resale_market = pd.merge(df1_flip, df2_flip[['ONS code', 'Area', 'Quarter', 'Year', 'Price']],
                          on=['ONS code', 'Area', 'Quarter', 'Year'], how='left')
df_resale_market

,ONS code,Area,Quarter,Houses Sold,Year,Price
0,E07000008,Cambridge,Q1,1438,1996,69950.0
1,E07000009,East Cambridgeshire,Q1,940,1996,56500.0
2,E07000010,Fenland,Q1,1214,1996,40000.0
3,E07000011,Huntingdonshire,Q1,2575,1996,54000.0
4,E06000031,Peterborough,Q1,2212,1996,40000.0
...,...,...,...,...,...,...
555,E07000010,Fenland,Q4,1660,2015,150000.0
556,E07000011,Huntingdonshire,Q4,2938,2015,215000.0
557,E06000031,Peterborough,Q4,2488,2015,148000.0
558,E07000012,South Cambridgeshire,Q4,2151,2015,315000.0


In [167]:
# Combine data frames df3 and df4 for primary market
df_primary_market = pd.merge(df3_flip, df4_flip[['ONS code', 'Area', 'Quarter', 'Year', 'Price']],
                          on=['ONS code', 'Area', 'Quarter', 'Year'], how='left')
df_primary_market

,ONS code,Area,Quarter,Houses Sold,Year,Price
0,E07000008,Cambridge,Q1,169,1996,90000.0
1,E07000009,East Cambridgeshire,Q1,324,1996,69950.0
2,E07000010,Fenland,Q1,252,1996,47950.0
3,E07000011,Huntingdonshire,Q1,589,1996,83000.0
4,E06000031,Peterborough,Q1,266,1996,50000.0
...,...,...,...,...,...,...
555,E07000010,Fenland,Q4,78,2015,152500.0
556,E07000011,Huntingdonshire,Q4,281,2015,197000.0
557,E06000031,Peterborough,Q4,501,2015,179995.0
558,E07000012,South Cambridgeshire,Q4,312,2015,345000.0


In [168]:
# Write for me python code that for each unique area name in my column in df - 'Area' prepare a plot with values of 'Price' on y ax and year and quater in ascending sequence on an axis x. View on my df is attached.

### Price trends for all districts over the years

### - for each quarter

In [176]:
df_primary_market['Year_Quarter'] = df_primary_market['Year'].astype(str) + ' ' + df_primary_market['Quarter']
df_primary_market.sort_values(by='Year_Quarter', inplace=True)

areas = df_primary_market['Area'].unique()


for area in areas:
    area_data = df_primary_market[df_primary_market['Area'] == area]
    
    # Extract only the year part from 'Year_Quarter' for ticks
    labels = area_data['Year_Quarter'].tolist()
    year_ticks = [label for index, label in enumerate(labels) if index % 4 == 0] 
    fig = px.line(area_data, x='Year_Quarter', y='Price', 
                  title=f'Price Trend for {area}', 
                  labels={'Year_Quarter': 'Year and Quarter', 'Price': 'Price'},
                  markers=True,
                  color_discrete_sequence=['green']) 

    fig.update_xaxes(tickvals=[labels.index(tick) for tick in year_ticks], ticktext=[tick.split()[0] for tick in year_ticks])
    
    fig.update_layout(
        xaxis_title='Year and Quarter',
        yaxis_title='Price',
        xaxis=dict(tickangle=45),
        template='plotly_white',
        autosize=True
    )
    
    fig.show()

### - average for each year

In [177]:
df_primary_market['Year'] = df_primary_market['Year_Quarter'].str.split().str[0]

# Group by 'Year' and 'Area' to calculate average price per year per area
yearly_avg = df_primary_market.groupby(['Year', 'Area']).agg({'Price': 'mean'}).reset_index()
areas = yearly_avg['Area'].unique()

n_rows = 2
n_cols = 4
fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=areas)


plot_row, plot_col = 1, 1

# uniform Y-axis range based on max and min of average prices

max_price = yearly_avg['Price'].max()
min_price = yearly_avg['Price'].min()

for i, area in enumerate(areas, start=1):
    area_data = yearly_avg[yearly_avg['Area'] == area]
    trace = go.Scatter(x=area_data['Year'], y=area_data['Price'], 
                       mode='lines+markers', name=area, 
                       line=dict(color='green'))
    
    fig.add_trace(trace, row=plot_row, col=plot_col)
    
    plot_col += 1
    if plot_col > n_cols: 
        plot_row += 1
        plot_col = 1
    if plot_row > n_rows and plot_col > n_cols:
        break

fig.update_yaxes(range=[min_price, max_price], nticks=10)

fig.update_layout(height=800, width=1200, title_text="Yearly Average Price Trends by District",
                  showlegend=False, template='plotly_white', title_x=0.5)

fig.show()

#Saving to HTML
fig.write_html('full_figure.html')


To have a general picture of prices and their changes over the years, price trends for all districts are presented. In the case of the 'Cambridge' district, price changes are the most rapid, and the smallest differences can be observed in the case of the 'Fenland' district.